In [ ]:
# 영업이익
# DB : code / date / term / profit
# DB : 코드 / 연도 / 기간 / 영업이익
# API : (stock_code) / bizYear / term / enpBzopPft

In [42]:
# crno 추출

import pandas as pd
import requests

test = pd.read_xml('CORPCODE.xml')
test

,corp_code,corp_name,stock_code,modify_date
0,434003,다코,NaN,20170630
1,434456,일산약품,NaN,20170630
2,430964,굿앤엘에스,NaN,20170630
3,432403,한라판지,NaN,20170630
4,388953,크레디피아제이십오차유동화전문회사,NaN,20170630
...,...,...,...,...
96965,151571,청림실업,NaN,20221114
96966,1143889,에이치엠지하우징,NaN,20221114
96967,1359578,성남대장피에프브이,NaN,20221114
96968,1002944,스마트에프앤디,NaN,20221114


In [43]:
# 005380 현대차 / 035420 NAVER / 068270 셀트리온 / 005930 삼성전자 / 000270 기아 / 
# 373220 LG에너지솔루션 / 207940 삼성바이오로직스 / 000660 SK하이닉스 / 006400 삼성SDI / 051910 LG화학
kospi_list = ["005380","035420","068270","005930","000270","373220","207940","000660","006400","051910"]

In [44]:
api_key_list={}

with open('api_key.txt', 'r') as file:
    api_keys = file.readlines()
    for api_key in api_keys:
        key, value = api_key.rstrip().split(':')
        api_key_list[key] = value
        # print(key, value)

crtfc_key = api_key_list['crtfc_key']
serviceKey = api_key_list['serviceKey']

In [46]:
name_list = []

for i in kospi_list:
    temp = test.loc[test['stock_code']==int(i)]
    code = temp['corp_code'].values[0]
    company = temp['corp_name'].values[0]
    print(company,f'{code:08d}')
    crtfc_key = crtfc_key # dart api key
    corp_code = f'{code:08d}'
    url = 'https://opendart.fss.or.kr/api/company.json?crtfc_key='+crtfc_key+'&corp_code=' + corp_code
    result=requests.get(url)
    print(result.json()['jurir_no'],'\n')

    name = {'stock_code':str(f'{int(i):06d}'), 'name':company, 'crno':result.json()['jurir_no']} # 'code':code, 

    name_list.append(name)

name_df = pd.DataFrame(name_list)
name_df

현대자동차 00164742
1101110085450 

NAVER 00266961
1101111707178 

셀트리온 00413046
1350110034038 

삼성전자 00126380
1301110006246 

기아 00106641
1101110037998 

LG에너지솔루션 01515323
1101117701356 

삼성바이오로직스 00877059
1201110566317 

SK하이닉스 00164779
1344110001387 

삼성SDI 00126362
1101110394174 

LG화학 00356361
1101112207995 



,stock_code,name,crno
0,005380,현대자동차,1101110085450
1,035420,NAVER,1101111707178
2,068270,셀트리온,1350110034038
3,005930,삼성전자,1301110006246
4,000270,기아,1101110037998
5,373220,LG에너지솔루션,1101117701356
6,207940,삼성바이오로직스,1201110566317
7,000660,SK하이닉스,1344110001387
8,006400,삼성SDI,1101110394174
9,051910,LG화학,1101112207995


In [ ]:
name_df.to_csv('stock_code.csv', index=False)

In [ ]:
corporate_code_list=[]

for i in kospi_list:
    temp = test.loc[test['stock_code']==int(i)]
    code = temp['corp_code'].values[0]
    company = temp['corp_name'].values[0]
    # print(company,f'{code:08d}')
    crtfc_key = crtfc_key # dart api key
    corp_code = f'{code:08d}'
    url = 'https://opendart.fss.or.kr/api/company.json?crtfc_key='+crtfc_key+'&corp_code=' + corp_code
    result=requests.get(url)
    corporate_code = result.json()['jurir_no']
    corporate_code_list.append(corporate_code)

In [ ]:
# 영업이익
# DB : code / date / term / profit
# API : (stock_code) / bizYear / term / enpBzopPft

# serviceKey = 공공데이터포털 인증키
serviceKey = serviceKey

# 샘플코드

import requests

url = 'http://apis.data.go.kr/1160100/service/GetFinaStatInfoService/getSummFinaStat'
params ={'serviceKey' : serviceKey, 'numOfRows' : '1', 'pageNo' : '1', 'resultType' : 'xml', 'crno' : '1746110000741', 'bizYear' : '2019' }

response = requests.get(url, params=params)
print(response.content)

In [ ]:
import requests
import json

serviceKey = serviceKey
# 20120101 / 20221226

bizYear = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022'] # '2015'부터 출력 / '2022' 없음

for crno in corporate_code_list:
    for year in bizYear:
        try: 
            url = 'http://apis.data.go.kr/1160100/service/GetFinaStatInfoService/getSummFinaStat'
            params ={'serviceKey' : serviceKey, 'numOfRows' : '10', 'pageNo' : '1', 'resultType' : 'json', 'crno' : crno, 'bizYear' : year }

            response = requests.get(url, params=params)
            contents = response.text
            # print(contents)

            json_ob = json.loads(contents)
            # print(json_ob)
            # print(type(json_ob)) #json타입 확인

            body = json_ob['response']['body']['items']['item']
            # print(body)

            df = pd.json_normalize(body)
            df = df[['bizYear', 'crno', 'enpBzopPft']]
            # print(df)

            # df.to_csv('df_test_profit.csv', mode='a') # mode='a' : 누적

        except:
            pass

In [2]:
df_edit = pd.read_csv('df_test_profit.csv')
df_edit

,Unnamed: 0,bizYear,crno,enpBzopPft
0,0.0,2015,1101110085450,6357906000000
1,1.0,2015,1101110085450,4267256000000
2,NaN,bizYear,crno,enpBzopPft
3,0.0,2016,1101110085450,5193500000000
4,1.0,2016,1101110085450,2699482000000
...,...,...,...,...
180,0.0,2020,1101112207995,1798159000000
181,1.0,2020,1101112207995,1114437000000
182,NaN,bizYear,crno,enpBzopPft
183,0.0,2021,1101112207995,5025466000000


In [3]:
df_edit.rename(columns={'Unnamed: 0':'기간'}, inplace=True)

In [4]:
df_edit = df_edit.dropna() # NaN값 있는 행 제거

In [5]:
# 해당 연도에 0만 있는 경우 : 0 -> 1로 변경

# 기간 컬럼 0:반기, 1:정기, term으로 변경 후 기간 컬럼 drop
df_edit.loc[df_edit['기간'].astype(str).str.contains("0.0"), 'term']='반기'
df_edit.loc[df_edit['기간'].astype(str).str.contains("1.0"), 'term']='정기'

df_edit.drop('기간', axis=1, inplace=True)

df_edit

c:\Users\SUN\anaconda3\envs\sun\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
c:\Users\SUN\anaconda3\envs\sun\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\Users\SUN\anaconda3\envs\sun\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the

,bizYear,crno,enpBzopPft,term
0,2015,1101110085450,6357906000000,반기
1,2015,1101110085450,4267256000000,정기
3,2016,1101110085450,5193500000000,반기
4,2016,1101110085450,2699482000000,정기
6,2017,1101110085450,4574667000000,반기
...,...,...,...,...
178,2019,1101112207995,582425000000,정기
180,2020,1101112207995,1798159000000,반기
181,2020,1101112207995,1114437000000,정기
183,2021,1101112207995,5025466000000,반기


In [6]:
df_edit.columns.tolist()
df_edit = df_edit[['bizYear', 'term', 'crno', 'enpBzopPft']] # 컬럼 순서 변경
df_edit

,bizYear,term,crno,enpBzopPft
0,2015,반기,1101110085450,6357906000000
1,2015,정기,1101110085450,4267256000000
3,2016,반기,1101110085450,5193500000000
4,2016,정기,1101110085450,2699482000000
6,2017,반기,1101110085450,4574667000000
...,...,...,...,...
178,2019,정기,1101112207995,582425000000
180,2020,반기,1101112207995,1798159000000
181,2020,정기,1101112207995,1114437000000
183,2021,반기,1101112207995,5025466000000


In [7]:
df_edit.to_csv('enpBzopPft.csv', index=False)

In [8]:
df1 = pd.read_csv('stock_code.csv', dtype=str) # 종목코드때문에 dtype=str 로 가져옴
df2 = pd.read_csv('enpBzopPft.csv')

In [9]:
df1

,stock_code,name,crno
0,005380,현대자동차,1101110085450
1,035420,NAVER,1101111707178
2,068270,셀트리온,1350110034038
3,005930,삼성전자,1301110006246
4,000270,기아,1101110037998
5,373220,LG에너지솔루션,1101117701356
6,207940,삼성바이오로직스,1201110566317
7,000660,SK하이닉스,1344110001387
8,006400,삼성SDI,1101110394174
9,051910,LG화학,1101112207995


In [10]:
df2

,bizYear,term,crno,enpBzopPft
0,2015,반기,1101110085450,6357906000000
1,2015,정기,1101110085450,4267256000000
2,2016,반기,1101110085450,5193500000000
3,2016,정기,1101110085450,2699482000000
4,2017,반기,1101110085450,4574667000000
...,...,...,...,...
118,2019,정기,1101112207995,582425000000
119,2020,반기,1101112207995,1798159000000
120,2020,정기,1101112207995,1114437000000
121,2021,반기,1101112207995,5025466000000


In [20]:
df1['crno']=pd.to_numeric(df1['crno']) # Python int too large to convert to C long 오류 해결

df2['crno']=pd.to_numeric(df2['crno'])

In [12]:
# crno(동일 컬럼) 기준으로 병합
df_merge = pd.merge(df1, df2, on = 'crno', how = 'outer')
df_merge

,stock_code,name,crno,bizYear,term,enpBzopPft
0,005380,현대자동차,1101110085450,2015,반기,6357906000000
1,005380,현대자동차,1101110085450,2015,정기,4267256000000
2,005380,현대자동차,1101110085450,2016,반기,5193500000000
3,005380,현대자동차,1101110085450,2016,정기,2699482000000
4,005380,현대자동차,1101110085450,2017,반기,4574667000000
...,...,...,...,...,...,...
118,051910,LG화학,1101112207995,2019,정기,582425000000
119,051910,LG화학,1101112207995,2020,반기,1798159000000
120,051910,LG화학,1101112207995,2020,정기,1114437000000
121,051910,LG화학,1101112207995,2021,반기,5025466000000


In [13]:
# 컬럼 순서 변경
df_merge.columns.tolist()
df_merge = df_merge[['stock_code', 'name', 'bizYear', 'term', 'enpBzopPft']] # crno

In [14]:
# 영업이익
# DB : code / date / term / profit
# API : (stock_code) / bizYear / term / enpBzopPft

df_merge = df_merge.rename(columns={'stock_code':'code', 'bizYear':'date', 'enpBzopPft':'profit'})
df_merge

,code,name,date,term,profit
0,005380,현대자동차,2015,반기,6357906000000
1,005380,현대자동차,2015,정기,4267256000000
2,005380,현대자동차,2016,반기,5193500000000
3,005380,현대자동차,2016,정기,2699482000000
4,005380,현대자동차,2017,반기,4574667000000
...,...,...,...,...,...
118,051910,LG화학,2019,정기,582425000000
119,051910,LG화학,2020,반기,1798159000000
120,051910,LG화학,2020,정기,1114437000000
121,051910,LG화학,2021,반기,5025466000000


In [15]:
# 주식 이름 drop
df_merge = df_merge.drop(['name'], axis=1, inplace=False)
df_merge

,code,date,term,profit
0,005380,2015,반기,6357906000000
1,005380,2015,정기,4267256000000
2,005380,2016,반기,5193500000000
3,005380,2016,정기,2699482000000
4,005380,2017,반기,4574667000000
...,...,...,...,...
118,051910,2019,정기,582425000000
119,051910,2020,반기,1798159000000
120,051910,2020,정기,1114437000000
121,051910,2021,반기,5025466000000


In [16]:
df_merge.to_csv('profit.csv', index=False)

In [17]:
df3 = pd.read_csv("profit.csv", dtype='str')
df3

,code,date,term,profit
0,005380,2015,반기,6357906000000
1,005380,2015,정기,4267256000000
2,005380,2016,반기,5193500000000
3,005380,2016,정기,2699482000000
4,005380,2017,반기,4574667000000
...,...,...,...,...
118,051910,2019,정기,582425000000
119,051910,2020,반기,1798159000000
120,051910,2020,정기,1114437000000
121,051910,2021,반기,5025466000000


In [47]:
# 반기 데이터 drop
delete = df3[df3['term']=='반기'].index
df3 = df3.drop(delete)
df3

,code,date,term,profit
1,005380,2015,정기,4267256000000
3,005380,2016,정기,2699482000000
5,005380,2017,정기,2163447000000
7,005380,2018,정기,-59322000000
9,005380,2019,정기,1580164000000
...,...,...,...,...
114,051910,2017,정기,2293229000000
116,051910,2018,정기,1953618000000
118,051910,2019,정기,582425000000
120,051910,2020,정기,1114437000000


In [19]:
df3.to_csv("profit_final.csv", index=False)